In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, rand, floor

spark = SparkSession.builder \
    .appName("Exercicio Intro") \
    .master("local[*]") \
    .getOrCreate()

df_nomes = spark.read.csv("/home/jovyan/work/Nomes_aleatorios.txt", header=True, inferSchema=True)

df_nomes.show(10)

df_nomes = df_nomes.withColumnRenamed("nome", "Nomes")

df_nomes.printSchema()
df_nomes.show(10)

escolaridades = ["Fundamental", "Medio", "Superior"]
df_nomes = df_nomes.withColumn("Escolaridade", lit(escolaridades).getItem((rand() * 3).cast("int")))

paises = ["Argentina", "Brasil", "Chile", "Colômbia", "Equador", "Paraguai", "Peru", "Uruguai", "Venezuela"]
df_nomes = df_nomes.withColumn("Pais", lit(paises).getItem((rand() * len(paises)).cast("int")))

df_nomes = df_nomes.withColumn("AnoNascimento", (floor(rand() * (2010 - 1945 + 1)) + 1945).cast("int"))

df_nomes.show(10)


+-----------------+
|   Frances Bennet|
+-----------------+
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
|   Kara Mcelwaine|
+-----------------+
only showing top 10 rows

root
 |-- Frances Bennet: string (nullable = true)

+-----------------+
|   Frances Bennet|
+-----------------+
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
|   Kara Mcelwaine|
+-----------------+
only showing top 10 rows

+-----------------+------------+---------+-------------+
|   Frances Bennet|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|    Jamie Russell|    Superior| Paraguai|         1969|
|   Edward Kistler|    Superior|    Chile|         1993|
|    Sheila Maurer|       Medio|Venezuela|         

/usr/local/spark/python/pyspark/sql/column.py:460: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


In [5]:
df_select = df_nomes.filter(col("AnoNascimento") >= 2000)

df_select.show(10)

+-----------------+------------+---------+-------------+
|   Frances Bennet|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|Berniece Ornellas|    Superior|   Brasil|         2007|
|    Brian Farrell|    Superior|Venezuela|         2007|
|   Kara Mcelwaine| Fundamental| Paraguai|         2000|
|  Helen Blackwell|       Medio| Paraguai|         2002|
|   Amanda Gravitt|       Medio|   Brasil|         2003|
|  Jerry Chynoweth|       Medio| Colômbia|         2003|
|   Rosie Lovelady|    Superior|  Uruguai|         2004|
|      Donald Vogt|    Superior| Colômbia|         2004|
| Flossie Anderson|    Superior|Venezuela|         2007|
|    Lynne Dustman|    Superior|Venezuela|         2005|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [6]:
df_nomes.createOrReplaceTempView("nomes")

df_select_sql = spark.sql("SELECT * FROM nomes WHERE AnoNascimento >= 2000")

df_select_sql.show(10)


+-----------------+------------+---------+-------------+
|   Frances Bennet|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|Berniece Ornellas|    Superior|   Brasil|         2007|
|    Brian Farrell|    Superior|Venezuela|         2007|
|   Kara Mcelwaine| Fundamental| Paraguai|         2000|
|  Helen Blackwell|       Medio| Paraguai|         2002|
|   Amanda Gravitt|       Medio|   Brasil|         2003|
|  Jerry Chynoweth|       Medio| Colômbia|         2003|
|   Rosie Lovelady|    Superior|  Uruguai|         2004|
|      Donald Vogt|    Superior| Colômbia|         2004|
| Flossie Anderson|    Superior|Venezuela|         2007|
|    Lynne Dustman|    Superior|Venezuela|         2005|
+-----------------+------------+---------+-------------+
only showing top 10 rows



In [7]:
millennials_count = df_nomes.filter((col("AnoNascimento") >= 1980) & (col("AnoNascimento") <= 1994)).count()
print(f"Quantidade de Millennials: {millennials_count}")

Quantidade de Millennials: 2273619


In [8]:
millennials_count_sql = spark.sql("SELECT COUNT(*) AS Millennials FROM nomes WHERE AnoNascimento BETWEEN 1980 AND 1994")
millennials_count_sql.show()

+-----------+
|Millennials|
+-----------+
|    2273619|
+-----------+



In [10]:
df_nomes = df_nomes.withColumnRenamed("País", "Pais")

df_nomes.createOrReplaceTempView("nomes")

geracoes_query = """
SELECT 
    Pais,
    CASE 
        WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
        WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
        WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
        WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 'Geração Z'
    END AS Geracao,
    COUNT(*) AS Quantidade
FROM nomes
GROUP BY Pais, Geracao
ORDER BY Pais, Geracao
"""

df_geracoes = spark.sql(geracoes_query)

df_geracoes.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Argentina|Baby Boomers|    336336|
|Argentina|   Geração X|    252274|
|Argentina|   Geração Z|    269198|
|Argentina| Millennials|    252649|
|   Brasil|Baby Boomers|    336317|
|   Brasil|   Geração X|    252400|
|   Brasil|   Geração Z|    269712|
|   Brasil| Millennials|    251650|
|    Chile|Baby Boomers|    336133|
|    Chile|   Geração X|    252948|
|    Chile|   Geração Z|    269450|
|    Chile| Millennials|    253422|
| Colômbia|Baby Boomers|    336917|
| Colômbia|   Geração X|    252756|
| Colômbia|   Geração Z|    268989|
| Colômbia| Millennials|    252868|
|  Equador|Baby Boomers|    336813|
|  Equador|   Geração X|    252791|
|  Equador|   Geração Z|    269591|
|  Equador| Millennials|    252797|
+---------+------------+----------+
only showing top 20 rows

